# Tune a CNN on MNIST

This tutorial walks through using Ax to tune two hyperparameters (learning rate and momentum) for a PyTorch CNN on the MNIST dataset trained using SGD with momentum.


In [1]:
import torch
import numpy as np
from ax import (
    ParameterType,
    RangeParameter,
    SearchSpace,
    SimpleExperiment,
    modelbridge,
    models,
)
from ax.modelbridge.factory import Models
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate

init_notebook_plotting()

[INFO 04-29 13:01:50] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


In [2]:
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Load MNIST data
First, we need to load the MNIST data and partition it into training, validation, and test sets.

Note: this will download the dataset if necessary.

In [3]:
train_loader, valid_loader, test_loader = load_mnist()

## 2. Define function to optimize
In this tutorial, we want to optimize classification accuracy on the validation set as a function of the learning rate and momentum. The function takes in a parameterization (set of parameter values), computes the classification accuracy, and returns a dictionary of metric name ('accuracy') to a tuple with the mean and standard error.

In [4]:
def train_evaluate(
    parameterization,  # dict of parameter names to values of those parameters
    weight=None,  # required by the evaluation function signature
):
    net = train(
        train_loader=train_loader, 
        parameters=parameterization,
        dtype=dtype,
        device=device,
    )
    accuracy = evaluate(
        net=net,
        data_loader=valid_loader,
        dtype=dtype,
        device=device,
    )
    return {'accuracy': (accuracy, 0.0)}

## 3. Define the search space for the parameters
Here, we set the bounds on the learning rate and momentum and set the parameter space for the learning rate to be on a log scale.

In [5]:
search_space = SearchSpace(parameters=[
    RangeParameter(
        name='lr', parameter_type=ParameterType.FLOAT, lower=1e-6, upper=0.4, log_scale=True
    ),
    RangeParameter(
        name='momentum', parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0,
    )
])

## 4. Set up experiment
Finally, we set up a SimpleExperiment with our search space, evaluation function, and objective.

In [6]:
exp = SimpleExperiment(
    name='tune_cnn',
    search_space=search_space,
    evaluation_function=train_evaluate,
    objective_name='accuracy',
)

## 5. Set up optimization loop

Run the optimization using the settings defined on the experiment. We will create 5 random sobol points for exploration followed by 15 points generated using the GPEI optimizer.

In [7]:
print(f"Running Sobol initialization trials...")
sobol = Models.SOBOL(exp.search_space)
for i in range(5):
    exp.new_trial(generator_run=sobol.gen(1))
    
for i in range(15):
    print(f"Running GP+EI optimization trial {i+1}/15...")
    # Reinitialize GP+EI model at each step with updated data.
    gpei = Models.GPEI(experiment=exp, data=exp.eval())
    batch = exp.new_trial(generator_run=gpei.gen(1))

Running Sobol initialization trials...
Running GP+EI optimization trial 1/15...
Running GP+EI optimization trial 2/15...
Running GP+EI optimization trial 3/15...
Running GP+EI optimization trial 4/15...
Running GP+EI optimization trial 5/15...
Running GP+EI optimization trial 6/15...
Running GP+EI optimization trial 7/15...
Running GP+EI optimization trial 8/15...
Running GP+EI optimization trial 9/15...
Running GP+EI optimization trial 10/15...
Running GP+EI optimization trial 11/15...
Running GP+EI optimization trial 12/15...
Running GP+EI optimization trial 13/15...
Running GP+EI optimization trial 14/15...
Running GP+EI optimization trial 15/15...


## 6. Plot response surface

Contour plot showing classification accuracy as a function of the two hyperparameters.

The black squares show points that we have actually run, notice how they are clustered in the optimal region.

In [8]:
render(plot_contour(model=gpei, param_x='lr', param_y='momentum', metric_name='accuracy'))

## 7. Plot best objective as function of the iteration

Show the model accuracy improving as we identify better hyperparameters.

In [9]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean for trial in exp.trials.values()]])
best_objective_plot = optimization_trace_single_method(
        y=np.maximum.accumulate(best_objectives, axis=1),
        title="Model performance vs. # of iterations",
        ylabel="Classification Accuracy",
)
render(best_objective_plot)

## 8. Train CNN with best hyperparameters and evaluate on test set

In [10]:
data = exp.fetch_data()
df = data.df
best_arm_name = df.arm_name[df['mean'] == df['mean'].max()].values[0]
best_arm = exp.arms_by_name[best_arm_name]
best_arm

Arm(name='17_0', parameters={'lr': 0.002380678340420389, 'momentum': 0.1537332495694931})

In [11]:
net = train(
    train_loader=train_loader, 
    parameters=best_arm.parameters,
    dtype=dtype,
    device=device,
)
test_accuracy = evaluate(
    net=net,
    data_loader=test_loader,
    dtype=dtype,
    device=device,
)

In [12]:
print(f"Classification Accuracy (test set): {round(test_accuracy*100, 2)}%")

Classification Accuracy (test set): 95.22%
